In [1]:
%load_ext autoreload

In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import numpy as np

/Users/matsalexander/.pyenv/versions/3.10.13/envs/gluon/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [37]:
%autoreload
import sys
sys.path.append('../')
from pipeline import Pipin
pipin = Pipin()

Current working directory: /Users/matsalexander/Desktop/Forest Gump/mats/gluon


In [40]:
df0 = pipin.get_combined_datasets(consecutive_threshold=10)

KeyError: 'date_calc'

In [28]:
df0
# df, df_tune = pipin.split_train_tune(df0)

,time,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,...,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,estimated,observed,A,B,C,pv_measurement
0,2019-06-02 22:00:00,7.700,1.22825,1728.949951,0.000000,0.000,1728.949951,0.0,280.299988,0.000,...,3.600,-3.575,-0.500,0.0,0.0,1.0,1,0,0,0.00
1,2019-06-02 23:00:00,7.700,1.22350,1689.824951,0.000000,0.000,1689.824951,0.0,280.299988,0.000,...,3.350,-3.350,0.275,0.0,0.0,1.0,1,0,0,0.00
2,2019-06-03 00:00:00,7.875,1.21975,1563.224976,0.000000,0.000,1563.224976,0.0,280.649994,0.000,...,3.050,-2.950,0.750,0.0,0.0,1.0,1,0,0,0.00
3,2019-06-03 01:00:00,8.425,1.21800,1283.425049,208.649994,0.750,1283.425049,0.0,281.674988,0.300,...,2.725,-2.600,0.875,0.0,0.0,1.0,1,0,0,0.00
4,2019-06-03 02:00:00,8.950,1.21800,1003.500000,32468.150391,23.100,1003.500000,0.0,282.500000,11.975,...,2.550,-2.350,0.925,0.0,0.0,1.0,1,0,0,19.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48027,2023-04-30 19:00:00,4.400,1.27550,1456.574951,84010.148438,4.175,551.224976,0.0,272.024994,2.775,...,4.075,3.600,1.875,0.0,1.0,0.0,0,0,1,50.96
48028,2023-04-30 20:00:00,4.400,1.27850,1476.349976,2206.800049,0.000,564.099976,0.0,271.950012,0.000,...,3.600,2.950,2.125,0.0,1.0,0.0,0,0,1,2.94
48029,2023-04-30 21:00:00,4.400,1.27900,1516.300049,0.000000,0.000,578.700012,0.0,271.899994,0.000,...,3.600,2.625,2.400,0.0,1.0,0.0,0,0,1,0.00
48030,2023-04-30 22:00:00,4.400,1.27975,1240.599976,0.000000,0.000,551.500000,0.0,271.950012,0.000,...,3.275,2.325,2.325,0.0,1.0,0.0,0,0,1,-0.00


In [29]:
# train, tune = TabularDataset(df), TabularDataset(df_tune)
train = TabularDataset(df0)

In [30]:
label="pv_measurement"
eval_metric="mean_absolute_error"

In [31]:
predictor1 = TabularPredictor(label=label, eval_metric=eval_metric).fit(
    train,
    presets='best_quality', 
    # tuning_data = tune,
    # use_bag_holdout=True,
    )

No path specified. Models will be saved in: "AutogluonModels/ag-20231106_133047/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231106_133047/"
AutoGluon Version:  0.8.2
Python Version:     3.10.13
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 22.5.0: Thu Jun  8 22:22:20 PDT 2023; root:xnu-8796.121.3~7/RELEASE_ARM64_T6000
Disk Space Avail:   723.58 GB / 994.66 GB (72.7%)
Train Data Rows:    48032
Train Data Columns: 51
Label Column: pv_measurement
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (5733.42, -0.0, 547.45889, 997.93177)
	If 'regression' is not the correct problem_type, please manually specify the problem_type param

In [32]:
test_data = pipin.get_combined_test_data()
test_data = TabularDataset(test_data)
test_data.shape


/Users/matsalexander/Desktop/Forest Gump/mats/gluon/../pipeline.py:253: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  return df.groupby(pd.Grouper(key=date_column, freq="1H")
/Users/matsalexander/Desktop/Forest Gump/mats/gluon/../pipeline.py:253: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  return df.groupby(pd.Grouper(key=date_column, freq="1H")
/Users/matsalexander/Desktop/Forest Gump/mats/gluon/../pipeline.py:253: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only col

(2160, 54)

In [33]:

pred = pd.DataFrame(predictor1.predict(test_data))

negatives_pred = pred[pred["pv_measurement"] < 0]
negatives_pred.shape

(99, 1)

In [34]:
from utils import get_unique_filename
final_prediction = pipin.post_processing(pred, prediction_column="pv_measurement")
final_prediction.to_csv(get_unique_filename('big_gluon__with_pipe'))

In [38]:
diff = pipin.compare_mae(final_prediction)
diff

371.5684770678928

In [39]:
predictor1.evaluate(train, silent=True)

{'mean_absolute_error': -84.1461711536094,
 'root_mean_squared_error': -209.8110124439562,
 'mean_squared_error': -44020.660942757946,
 'r2': 0.9557957624239488,
 'pearsonr': 0.985642826497674,
 'median_absolute_error': -9.06751907348634}